In [7]:
import cv2
import mediapipe as mp
import os
import numpy as np
import time

# Initialize MediaPipe Holistic model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Create directories for data storage
DATA_PATH = 'sign_language_data'
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# Actions (signs) we want to detect
actions = ['hello', 'my', 'name', 'is','i', 'study', 'in', 'department','am']
no_sequences = 50 # Number of videos for each action
sequence_length = 40  # Number of frames in each video

def create_folders_for_action(action):
    """Create folders for a specific action"""
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

def extract_keypoints(results):
    """Extract keypoints from MediaPipe Holistic results"""
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

def show_action_selection():
    """Display action selection menu and return selected action"""
    # Create a black image
    img = np.zeros((500, 700, 3), np.uint8)
    
    # Add title
    cv2.putText(img, "Select Action to Record:", (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display all actions with corresponding numbers
    for i, action in enumerate(actions):
        cv2.putText(img, f"{i+1}. {action}", (50, 100 + i*50), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    # Add instructions
    cv2.putText(img, "Press number key to select action", (50, 400), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 1)
    cv2.putText(img, "Press ESC to quit", (50, 450), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)
    
    cv2.imshow("Action Selection", img)
    
    while True:
        key = cv2.waitKey(0)
        
        # Check for ESC key (27 is ASCII for ESC)
        if key == 27:
            cv2.destroyAllWindows()
            return None
        
        # Check for number keys 1-7
        if 49 <= key <= 57:  # ASCII codes for 1-9
            action_index = key - 49  # Convert to 0-based index
            if action_index < len(actions):
                cv2.destroyAllWindows()
                return actions[action_index]
        
        # Also support numpad numbers
        if 97 <= key <= 105:  # ASCII codes for numpad 1-9
            action_index = key - 97  # Convert to 0-based index
            if action_index < len(actions):
                cv2.destroyAllWindows()
                return actions[action_index]

def collect_data_for_action(action):
    """Collect sign language data for a specific action"""
    cap = cv2.VideoCapture(0)
    
    # Set mediapipe model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        
        # Loop through sequences (videos)
        for sequence in range(no_sequences):
            # Show preparation message
            prep_img = np.zeros((300, 700, 3), np.uint8)
            cv2.putText(prep_img, f"Prepare to record: {action}", (50, 100), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(prep_img, f"Sequence {sequence + 1}/{no_sequences}", (50, 150), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.putText(prep_img, "Press any key to start recording", (50, 200), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 1)
            cv2.imshow("Preparation", prep_img)
            cv2.waitKey(0)
            cv2.destroyWindow("Preparation")
            
            # Loop through video length (sequence length)
            for frame_num in range(sequence_length):
                # Read feed
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Make detections
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(image)
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                
                # Draw landmarks
                mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
                mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                
                # Display recording status
                cv2.putText(image, f'Recording: {action}', (15, 20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.putText(image, f'Sequence: {sequence + 1}/{no_sequences}', (15, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.putText(image, f'Frame: {frame_num + 1}/{sequence_length}', (15, 80), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1, cv2.LINE_AA)
                
                # Show to screen
                cv2.imshow('Recording', image)
                
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                # Check for ESC key to quit
                if cv2.waitKey(10) & 0xFF == 27:
                    cap.release()
                    cv2.destroyAllWindows()
                    return False
            
            # Show completion message
            complete_img = np.zeros((300, 700, 3), np.uint8)
            cv2.putText(complete_img, f"Completed sequence {sequence + 1}", (50, 100), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(complete_img, "Press any key to continue", (50, 150), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1)
            cv2.putText(complete_img, "ESC to stop recording", (50, 200), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)
            cv2.imshow("Sequence Complete", complete_img)
            
            # Check if user wants to continue or quit
            key = cv2.waitKey(0)
            cv2.destroyWindow("Sequence Complete")
            if key == 27:  # ESC key
                cap.release()
                cv2.destroyAllWindows()
                return False
    
    cap.release()
    cv2.destroyAllWindows()
    return True

def main():
    """Main function to run the data collection"""
    while True:
        # Show action selection menu
        selected_action = show_action_selection()
        
        # If no action selected (ESC pressed), exit
        if selected_action is None:
            print("Exiting data collection...")
            break
        
        # Create folders for the selected action
        create_folders_for_action(selected_action)
        
        # Start data collection for selected action


        
        print(f"Starting data collection for: {selected_action}")
        success = collect_data_for_action(selected_action)
        
        if not success:
            print(f"Data collection stopped for: {selected_action}")
            break
        
        print(f"Completed data collection for: {selected_action}")

if __name__ == "__main__":
    main()

Exiting data collection...
